In [2]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pandas as pd
dfet=pd.read_csv('Everyone10+.csv')
dfet

,Name,Platform,Genre,Rating,Poster
0,LEGO Batman: The Videogame,Wii,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,LEGO Batman: The Videogame,DS,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,Kung Fu Panda,X360,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,Skylanders: Spyro's Adventure,Wii,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,LEGO Star Wars II: The Original Trilogy,PS2,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,LEGO Star Wars: The Complete Saga,X360,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,Skylanders Giants,Wii,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,LEGO Harry Potter: Years 1-4,Wii,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,The Legend of Zelda: Majora's Mask 3D,3DS,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,LEGO Indiana Jones: The Original Adventures,Wii,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


In [3]:
dfet['combined_features']=dfet['Genre']+dfet['Rating']
dfet['combined_features']

0        ActionE10+
1        ActionE10+
2        ActionE10+
3        ActionE10+
4        ActionE10+
5        ActionE10+
6        ActionE10+
7        ActionE10+
8        ActionE10+
9        ActionE10+
10       ActionE10+
11       ActionE10+
12       ActionE10+
13       ActionE10+
14       ActionE10+
15       ActionE10+
16       ActionE10+
17       ActionE10+
18       ActionE10+
19       ActionE10+
20    AdventureE10+
21    AdventureE10+
22    AdventureE10+
23    AdventureE10+
24    AdventureE10+
25    AdventureE10+
26    AdventureE10+
27    AdventureE10+
28    AdventureE10+
29    AdventureE10+
30    AdventureE10+
31    AdventureE10+
32       ActionE10+
33    AdventureE10+
34    AdventureE10+
35    AdventureE10+
36    AdventureE10+
37    AdventureE10+
38    AdventureE10+
39    AdventureE10+
40    AdventureE10+
Name: combined_features, dtype: object

In [4]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(dfet['combined_features'])

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [6]:
knn = NearestNeighbors(n_neighbors=10, metric='cosine')  # n_neighbors=10 means top-10 similar games
knn.fit(cosine_sim)


NearestNeighbors(metric='cosine', n_neighbors=10)

In [7]:
def recommend_games(name):
    game_index = dfet[dfet['Name'].str.lower() == name.lower()].index[0]
    distance, indices = knn.kneighbors(cosine_sim[game_index].reshape(1, -1), n_neighbors=6)
    recommended_game_indices = indices.flatten()[1:]
    recommended_games = dfet.iloc[recommended_game_indices]
    return recommended_games['Name']

g = recommend_games('Kung Fu Panda')

# Print movie names without index
for m in g:
    print(m)


LEGO Batman: The Videogame
Kung Fu Panda
Skylanders: Spyro's Adventure
LEGO Star Wars II: The Original Trilogy
LEGO Star Wars: The Complete Saga


In [9]:
import pickle
pickle.dump(dfet, open('kdramat.pkl', 'wb'))
pickle.dump(knn, open('model.pkl', 'wb'))
pickle.load(open('kdramat.pkl', 'rb'))

,Name,Platform,Genre,Rating,Poster,combined_features
0,LEGO Batman: The Videogame,Wii,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",ActionE10+
1,LEGO Batman: The Videogame,DS,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",ActionE10+
2,Kung Fu Panda,X360,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",ActionE10+
3,Skylanders: Spyro's Adventure,Wii,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",ActionE10+
4,LEGO Star Wars II: The Original Trilogy,PS2,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",ActionE10+
5,LEGO Star Wars: The Complete Saga,X360,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",ActionE10+
6,Skylanders Giants,Wii,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",ActionE10+
7,LEGO Harry Potter: Years 1-4,Wii,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",ActionE10+
8,The Legend of Zelda: Majora's Mask 3D,3DS,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",ActionE10+
9,LEGO Indiana Jones: The Original Adventures,Wii,Action,E10+,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",ActionE10+
